In [1]:
import json

from mvodolagin_personal_imports import *

load_dotenv()

True

In [2]:
data_dir = Path(r"E:\Work\Personal\repos\web_scrapers\spiders\texttailor\data\carolhouse_spider")

In [3]:
for file in data_dir.glob("*.html"):
    with open(file, "r", encoding="utf-8") as f:
        html_text = f.read()
    break

In [ ]:
from readability import Document as readability_Document
import html2text
from trafilatura import extract

text_maker = html2text.HTML2Text()
text_maker.ignore_links = True

raw_data_path = Path(r"E:\Work\Personal\repos\web_scrapers\spiders\glossary_scraper\data")

raw_documents = []

for subdir in raw_data_path.glob("*"):
    source_name = subdir.stem
    common_elements, _ = populate_common_elements(subdir, threshold=0.25)
    for html_file in tqdm(list(subdir.glob("*.html"))):
        if html_file.stem == "start_page":
            continue
        page_name = html_file.stem.split("-", 1)[-1]
        with open(html_file, "r", encoding="utf-8") as f:
            html = f.read()

        modified_html = remove_common_elements(html, common_elements)
        doc = json.loads(extract(modified_html, output_format="json", include_comments=True))
        doc["page_name"] = page_name

        raw_documents.append(doc)


In [4]:
from trafilatura import extract

doc = json.loads(extract(html_text, output_format="json", include_comments=True))

In [5]:
doc

{'title': 'Pulaski Furniture Living Room Two Way Sliding Door Curio 20484',
 'author': None,
 'hostname': 'carolhouse.com',
 'date': '2024-01-01',
 'fingerprint': '966a2b6c5da7d07f',
 'id': None,
 'license': None,
 'comments': '',
 'raw_text': 'Pulaski Furniture Living Room Two Way Sliding Door Curio 20484 at Carol House Furniture This beautiful mirrored-back curio cabinet lets you display your collections: a source of pride, accomplishment, and fond memories. Give your cherished belongings the spotlight they deserve with this beautiful locking curio cabinet, fitted with a sliding beveled class door and locking mechanism for security. Five generously-sized glass shelves can be adjusted into nearly any configuration with top-to-bottom position holes every two inches, allowing you to customize the display to your liking. Along the framed edges, stunning rope-and-block molding frames and flatters the curio\'s contents, while two-setting halogen top-mounted lights provide an attractive, un

In [6]:
doc.keys()

dict_keys(['title', 'author', 'hostname', 'date', 'fingerprint', 'id', 'license', 'comments', 'raw_text', 'text', 'language', 'image', 'pagetype', 'source', 'source-hostname', 'excerpt', 'categories', 'tags'])

In [8]:
doc["source"]

'https://www.carolhouse.com/pulaski-furniture-two-way-sliding-door-curio/20484-707/iteminformation.aspx'

In [4]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html_text, "html.parser")

In [14]:
# Check which of the lowest-level tags have "Product Description" in them

for tag in soup.find_all():
    if "Product Description" in tag.text:
        print(tag.name)
        break



html


In [20]:
candidates = []

# Search for tags that contain text and bold elements
for tag in soup.find_all(string=True):
    parent = tag.parent
    if parent.name not in ['script', 'style'] and parent.find(['b', 'strong']):
        candidates.append(parent)

In [28]:
unique_candidates = list({c for c in candidates if not any([c.find_parent() in candidates])})

In [29]:
len(unique_candidates)

2

In [34]:
unique_candidates[0].name

'html'

In [35]:
def find_description_tags(html):
    soup = BeautifulSoup(html, 'html.parser')
    candidates = []

    # Search for tags that contain text and bold elements
    for tag in soup.find_all(text=True):
        parent = tag.parent
        if parent.name not in ['script', 'style'] and "Product Description" in tag:
            candidates.append(parent)

    # Narrow down to unique smallest candidates by filtering out parents of found tags
    unique_candidates = list({c for c in candidates if not any([c.find_parent() in candidates])})
    unique_candidates = [c for c in unique_candidates if c.name != "html"]

    # Further processing to ensure they match your product description criteria
    descriptions = [tag for tag in unique_candidates if 'product' in tag.text.lower()]

    return descriptions

In [36]:
description_tags = find_description_tags(html_text)
for tag in description_tags:
    print(tag, '\n')

<span class="ProductInfoSpanName ProductInfoSpanNameProductDescription">Product Description</span> 


C:\Users\Karganak\AppData\Local\Temp\ipykernel_38904\1672327430.py:6: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for tag in soup.find_all(text=True):


In [39]:
tag.next_sibling.next_sibling

<span class="ProductInfoSpanValue ProductInfoSpanValueProductDescription">This beautiful mirrored-back curio cabinet lets you display your collections: a source of pride, accomplishment, and fond memories. Give your cherished belongings the spotlight they deserve with this beautiful locking curio cabinet, fitted with a sliding beveled class door and locking mechanism for security. Five generously-sized glass shelves can be adjusted into nearly any configuration with top-to-bottom position holes every two inches, allowing you to customize the display to your liking. Along the framed edges, stunning rope-and-block molding frames and flatters the curio's contents, while two-setting halogen top-mounted lights provide an attractive, uncluttered view of your items. This curio cabinet is finished with glass sides and a fully mirrored back that allows you to see your items on display from every angle. A warm honey-brown finish on the cabinet's attractive veneers goes with any decor theme for e

In [5]:
from bs4 import BeautifulSoup
from collections import Counter


def is_lowest_level_tag(tag):
    # A tag is the lowest level if it does not contain other tags with text
    return not any(child for child in tag.children if child.name and child.get_text(strip=True))


def collect_lowest_level_tags_with_substantial_text(html):
    soup = BeautifulSoup(html, 'html.parser')
    tags_by_class = {}

    for tag in soup.find_all(True):
        if tag.has_attr('class') and is_lowest_level_tag(tag) and len(tag.get_text(strip=True)) >= 20:
            class_name = ' '.join(tag['class'])
            if class_name not in tags_by_class:
                tags_by_class[class_name] = []
            tags_by_class[class_name].append(tag)

    return tags_by_class


def collect_texts_by_class_from_tags(html):
    # Placeholder for the modified collect_lowest_level_tags_with_substantial_text function
    # This version is assumed to return a dict with class names as keys and lists of tag objects as values
    substantial_tags = collect_lowest_level_tags_with_substantial_text(html)

    texts_by_class = {}
    for class_name, tags in substantial_tags.items():
        # Initialize or extend the list of texts for the class name
        texts_by_class.setdefault(class_name, [])
        for tag in tags:
            texts_by_class[class_name].append(tag.get_text(strip=True))

    return texts_by_class


# Collect and print tags with substantial text
substantial_tag_counts = collect_lowest_level_tags_with_substantial_text(html_text)
texts_by_class = collect_texts_by_class_from_tags(html_text)


In [6]:
texts_by_class

{'shopbycolorLabel': ['Shiatsu Massage Chairs',
  'Bunk Beds & Loft Beds',
  'Home Theater Seating',
  'Shiatsu Massage Chairs',
  'Bassett Mirror Company',
  'Best Home Furnishings',
  'Carol House Design Collection',
  'Carol House Reclaimed Collection',
  'Hillsdale Kids and Teen',
  'Jonathan Louis International',
  'Legacy Classic Furniture',
  'Pacific Coast Lighting',
  'Perfect Sleeper® by Serta',
  'Rachael Ray Home by Legacy Classic Furniture',
  'Vaughan-Bassett Furniture Company'],
 'Label': ['Entertainment Centers'],
 'ProductImageDisclaimerParagraph ProductImageDisclaimerParagraphPhotoText': ['Upholstered furniture may be stocked in a different color or pattern than picture shows. Fabric in pictures may also be available by custom order.'],
 'viewallincollectionLink': ['View all of the items in the PFC Curios collection'],
 'ProductDescriptionHeading': ['Pulaski Furniture Living Room Two Way Sliding Door Curio 20484 at Carol House Furniture'],
 'popupForReqInfoFormFn omni

In [7]:
root_page_html = Path(
    r"C:\Users\Karganak\Downloads\Vivaldi downloads\Carol House Furniture _ Largest Selection Lowest Price.html")

In [8]:
with open(root_page_html, "r", encoding="utf-8") as f:
    root_page_text = f.read()

In [9]:
soup = BeautifulSoup(root_page_text, "html.parser")

In [10]:
# get all the links from within the html. Fully resolved, unique links only

def get_links_from_html(html):
    soup = BeautifulSoup(html, "html.parser")
    links = set()
    for link in soup.find_all("a"):
        if link.has_attr("href"):
            links.add(link["href"])
    return links

In [11]:
ll = get_links_from_html(root_page_text)

In [12]:
ll = [l for l in ll if l.startswith("/") and l != "/"]

In [13]:
ll

['/bedroom/cabinets/room-type.aspx',
 '/bedroom/room-type.aspx?features=baby%2fyouth',
 '/dining-room/stools/room-type.aspx',
 '/terms-conditions.inc',
 '/southern-motion/brand-type.aspx',
 '/designcenter.inc',
 '/american-drew/brand-type.aspx',
 '/homepage.aspx?logout=true',
 '/carol-house-design-collection/brand-type.aspx',
 '/office-furniture/desks/category-type.aspx',
 '/office-furniture/tables/category-type.aspx',
 '/living-room/chaises/room-type.aspx',
 '/brand/john-thomas/',
 '/in-home-design-staging.inc',
 '/art-furniture/brand-type.aspx',
 '/cooper-classics/brand-type.aspx',
 '/accessories/mirrors/department-type.aspx',
 '/winners-only/brand-type.aspx',
 '/entertainment.inc',
 '/bedroom/entertainment-centers/room-type.aspx',
 '/flexsteel/brand-type.aspx',
 '/riverside/brand-type.aspx',
 '/best-home-furnishings/brand-type.aspx',
 '/dining-room/cabinets/room-type.aspx',
 '/living-room/loveseats/room-type.aspx',
 '/howard-elliott/brand-type.aspx',
 '/bedroom/mirrors/room-type.asp

In [14]:
from mvodolagin_personal_imports.langchain_stuff import *

In [59]:
def get_prompt_for_links_list(ll):
    role_message = """You will be given a numbered list of internal links. Some of them link to item catalogues. Your task is to find the links that lead to item catalogues and return their numbers. Return a JSON that contains only the numbers."""

    example_input = """1. /clothing/men
    2. /about
    3. /deals/item-catalogue
    4. /contact-us"""

    example_output = """{{"catalogue_links": [1, 3]}}"""

    formatted_list = "\n".join([f"{i + 1}. {link}" for i, link in enumerate(ll)])

    messages = [("system", role_message), ("human", example_input), ("ai", example_output), ("human", formatted_list)]
    messages = [(role, trim_extra_whitespace(message)) for role, message in messages]

    full_prompt = ChatPromptTemplate.from_messages(messages)
    return full_prompt


In [68]:
def get_prompt_for_links_regexp(ll):
    role_message = """You will be presented with a numbered list of internal links. Your task is to analyze these links and derive a Python-style regular expression (regexp) pattern that could be used to identify links leading to item catalogues. The goal is to create a regexp template that matches as many catalogue links as possible without being overly broad. Return only the regexp pattern in a string format."""

    example_input = """1. /bedroom/cabinets/room-type.aspx
2. /terms-conditions.inc
3. /southern-motion/brand-type.aspx
4. /designcenter.inc
5. /office-furniture/desks/category-type.aspx
6. /homepage.aspx?logout=true
7. /living-room/chaises/room-type.aspx
8. /brand/john-thomas/
9. /in-home-design-staging.inc"""

    example_output = "\/(room-type|brand-type|category-type)\.aspx"

    formatted_list = "\n".join([f"{i + 1}. {link}" for i, link in enumerate(ll)])

    messages = [("system", role_message), ("human", example_input), ("ai", example_output), ("human", formatted_list)]
    messages = [(role, trim_extra_whitespace(message)) for role, message in messages]

    full_prompt = ChatPromptTemplate.from_messages(messages)
    return full_prompt


In [64]:
full_prompt = get_prompt_for_links_regexp(ll)

In [65]:
res_o = (full_prompt | basic_llm).invoke({})
res_o

AIMessage(content='\\/(room-type|brand-type|category-type)\\/?.*', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 2082, 'total_tokens': 2096}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}, id='run-6af0d042-df0c-4ca0-a7bd-4d7cb5e8a57d-0')

In [66]:
res_a = (full_prompt | basic_llm_claude).invoke({})
res_a

AIMessage(content='The regular expression pattern that could be used to identify links leading to item catalogues is:\n\n\\/((room|brand|category|department)-type)\\.aspx\n\nThis pattern matches links that contain either "room-type", "brand-type", "category-type", or "department-type" in the URL path, followed by ".aspx".', response_metadata={'id': 'msg_01KdWXBeYXbLC7faa1NavPvC', 'content': [ContentBlock(text='The regular expression pattern that could be used to identify links leading to item catalogues is:\n\n\\/((room|brand|category|department)-type)\\.aspx\n\nThis pattern matches links that contain either "room-type", "brand-type", "category-type", or "department-type" in the URL path, followed by ".aspx".', type='text')], 'model': 'claude-3-haiku-20240307', 'role': 'assistant', 'stop_reason': 'end_turn', 'stop_sequence': None, 'type': 'message', 'usage': Usage(input_tokens=2583, output_tokens=77)}, id='run-e5659ead-aa95-45a7-8656-96029f01e801-0')

In [69]:
res_o_mid = (full_prompt | mid_llm).invoke({})
res_o_mid

AIMessage(content='Based on the provided list of links, we can observe that the links leading to item catalogues have a consistent pattern where they include a specific type (room-type, brand-type, category-type, or department-type) followed by `.aspx`. To match these catalogue links and not the others, we can use the following regexp pattern:\n\n```\n\\/(room-type|brand-type|category-type|department-type)\\.aspx\n```\n\nThis pattern ensures that we match any link that ends with one of the specified types followed by `.aspx`, which seems to be the common format for catalogue links in the given list.', response_metadata={'token_usage': {'completion_tokens': 126, 'prompt_tokens': 2082, 'total_tokens': 2208}, 'model_name': 'gpt-4-0125-preview', 'system_fingerprint': 'fp_b77cb481ed', 'finish_reason': 'stop', 'logprobs': None}, id='run-fafe3e17-677e-472f-8c37-8c227aff35d1-0')

In [62]:
clipboard_copy(formatted_list)

In [29]:
numbers = safe_json_loads(res_a.content.split("\n\n")[-1])
to_join = [ll[i - 2] for i in numbers]
sample = random.sample(to_join, 10)
text = "\n".join(sample)
clipboard_copy(text)

In [27]:
formatted_list

'2. /bedroom/cabinets/room-type.aspx\n3. /bedroom/room-type.aspx?features=baby%2fyouth\n4. /dining-room/stools/room-type.aspx\n5. /terms-conditions.inc\n6. /southern-motion/brand-type.aspx\n7. /designcenter.inc\n8. /american-drew/brand-type.aspx\n9. /homepage.aspx?logout=true\n10. /carol-house-design-collection/brand-type.aspx\n11. /office-furniture/desks/category-type.aspx\n12. /office-furniture/tables/category-type.aspx\n13. /living-room/chaises/room-type.aspx\n14. /brand/john-thomas/\n15. /in-home-design-staging.inc\n16. /art-furniture/brand-type.aspx\n17. /cooper-classics/brand-type.aspx\n18. /accessories/mirrors/department-type.aspx\n19. /winners-only/brand-type.aspx\n20. /entertainment.inc\n21. /bedroom/entertainment-centers/room-type.aspx\n22. /flexsteel/brand-type.aspx\n23. /riverside/brand-type.aspx\n24. /best-home-furnishings/brand-type.aspx\n25. /dining-room/cabinets/room-type.aspx\n26. /living-room/loveseats/room-type.aspx\n27. /howard-elliott/brand-type.aspx\n28. /bedroom/

In [31]:
url_pattern = re.compile(
    r'^/[\w-]+(/[\w-]+)*(/[\w-]+(\.aspx|\.inc)?)?/?$'
)

for l in ll:
    if not url_pattern.match(l):
        print(l)

/bedroom/room-type.aspx?features=baby%2fyouth
/terms-conditions.inc
/designcenter.inc
/homepage.aspx?logout=true
/in-home-design-staging.inc
/entertainment.inc
/satisfactionguarantee.inc
/history.inc
/communityoutreach.inc
/freeconciergeservices.inc
/DisplayFlyer.aspx
/livingroom.inc
/careers.inc
/contactus.inc
/displayflyer.aspx
/visual-site-map.inc
/viewcart.aspx
/favorites.aspx
/aboutus.inc
/modifyaccount.aspx
/createaccount.aspx
/brands.inc
/WhatOurCustomersThink.inc
/furniturecare.inc
/bedroom.inc
/itembrowser.aspx?action=attributes&Category=Mattresses
/diningroom.inc
/accessories.inc
/ch-vintage-commercials.inc
/privacy-policy.inc
/contractsalesvolume.inc
/joinemail.inc
/homeoffice.inc
/login.aspx


In [32]:
texts_by_class

{'shopbycolorLabel': ['Shiatsu Massage Chairs',
  'Bunk Beds & Loft Beds',
  'Home Theater Seating',
  'Shiatsu Massage Chairs',
  'Bassett Mirror Company',
  'Best Home Furnishings',
  'Carol House Design Collection',
  'Carol House Reclaimed Collection',
  'Hillsdale Kids and Teen',
  'Jonathan Louis International',
  'Legacy Classic Furniture',
  'Pacific Coast Lighting',
  'Perfect Sleeper® by Serta',
  'Rachael Ray Home by Legacy Classic Furniture',
  'Vaughan-Bassett Furniture Company'],
 'Label': ['Entertainment Centers'],
 'ProductImageDisclaimerParagraph ProductImageDisclaimerParagraphPhotoText': ['Upholstered furniture may be stocked in a different color or pattern than picture shows. Fabric in pictures may also be available by custom order.'],
 'viewallincollectionLink': ['View all of the items in the PFC Curios collection'],
 'ProductDescriptionHeading': ['Pulaski Furniture Living Room Two Way Sliding Door Curio 20484 at Carol House Furniture'],
 'popupForReqInfoFormFn omni

In [37]:
from langchain_core.prompt_values import ChatPromptValue

role_message = """Your task is to determine which of the html tags contain description of the item on the page in a retail marketplace. You will be given a dictionary of tags and the text they contain. Return a JSON that contains the tags that contain the description of the item. Write only the tags."""

human_template = "{formatted_data}"

full_prompt = ChatPromptTemplate.from_messages([("system", role_message), ("human", human_template)])

In [38]:
res_o = (full_prompt | basic_llm).invoke({"formatted_data": json.dumps(texts_by_class, indent=4)})
res_o

AIMessage(content='```json\n[\n    "ProductDescriptionHeading",\n    "ProductInfoSpanValue ProductInfoSpanValueProductDescription",\n    "ProductInfoSpanValue ProductInfoSpanValueDescription"\n]\n```', response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 1401, 'total_tokens': 1438}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}, id='run-adf53a56-215b-4121-9404-320dd532a0cf-0')

In [39]:
res_a = (full_prompt | basic_llm_claude).invoke({"formatted_data": json.dumps(texts_by_class, indent=4)})
res_a

AIMessage(content='Based on the given dictionary of tags and text, the tags that contain the description of the item are:\n\n[\n  "ProductDescriptionHeading",\n  "ProductInfoSpanValue ProductInfoSpanValueProductDescription",\n  "ProductInfoSpanValue ProductInfoSpanValueCollectionFeatures",\n  "ProductInfoSpanValue ProductInfoSpanValueFeatures",\n  "ProductInfoSpanValue ProductInfoSpanValueFinish",\n  "ProductInfoSpanValue ProductInfoSpanValueFrameConstruction",\n  "ProductInfoSpanValue ProductInfoSpanValueDescription",\n  "ProductInfoSpanValue ProductInfoSpanValueType"\n]', response_metadata={'id': 'msg_017A39B6oPZhcaXkq8bLEJzj', 'content': [ContentBlock(text='Based on the given dictionary of tags and text, the tags that contain the description of the item are:\n\n[\n  "ProductDescriptionHeading",\n  "ProductInfoSpanValue ProductInfoSpanValueProductDescription",\n  "ProductInfoSpanValue ProductInfoSpanValueCollectionFeatures",\n  "ProductInfoSpanValue ProductInfoSpanValueFeatures",\n  

In [40]:
res_a_smart = (full_prompt | smart_llm_claude).invoke({"formatted_data": json.dumps(texts_by_class, indent=4)})
res_a_smart

AIMessage(content='The tags that contain the description of the item are:\n\n[\n  "ProductInfoSpanValue ProductInfoSpanValueProductDescription",\n  "ProductInfoSpanValue ProductInfoSpanValueCollectionFeatures",\n  "ProductInfoSpanValue ProductInfoSpanValueDescription"\n]', response_metadata={'id': 'msg_01VZxT5MULV9DYFhV5qGK4EJ', 'content': [ContentBlock(text='The tags that contain the description of the item are:\n\n[\n  "ProductInfoSpanValue ProductInfoSpanValueProductDescription",\n  "ProductInfoSpanValue ProductInfoSpanValueCollectionFeatures",\n  "ProductInfoSpanValue ProductInfoSpanValueDescription"\n]', type='text')], 'model': 'claude-3-opus-20240229', 'role': 'assistant', 'stop_reason': 'end_turn', 'stop_sequence': None, 'type': 'message', 'usage': Usage(input_tokens=1735, output_tokens=67)}, id='run-30ed8da5-a534-4b97-8d76-2d406aa513bb-0')

In [41]:
print(res_a_smart.content)

The tags that contain the description of the item are:

[
  "ProductInfoSpanValue ProductInfoSpanValueProductDescription",
  "ProductInfoSpanValue ProductInfoSpanValueCollectionFeatures",
  "ProductInfoSpanValue ProductInfoSpanValueDescription"
]


In [42]:
print(res_a.content)

Based on the given dictionary of tags and text, the tags that contain the description of the item are:

[
  "ProductDescriptionHeading",
  "ProductInfoSpanValue ProductInfoSpanValueProductDescription",
  "ProductInfoSpanValue ProductInfoSpanValueCollectionFeatures",
  "ProductInfoSpanValue ProductInfoSpanValueFeatures",
  "ProductInfoSpanValue ProductInfoSpanValueFinish",
  "ProductInfoSpanValue ProductInfoSpanValueFrameConstruction",
  "ProductInfoSpanValue ProductInfoSpanValueDescription",
  "ProductInfoSpanValue ProductInfoSpanValueType"
]


In [43]:
print(res_o.content)

```json
[
    "ProductDescriptionHeading",
    "ProductInfoSpanValue ProductInfoSpanValueProductDescription",
    "ProductInfoSpanValue ProductInfoSpanValueDescription"
]
```


In [44]:
clipboard_copy(texts_by_class)

In [47]:
classes_to_parse = ["ProductDescriptionHeading",
                    "ProductInfoSpanValue ProductInfoSpanValueProductDescription",
                    "ProductInfoSpanValue ProductInfoSpanValueCollectionFeatures"]


def parse_description(html, classes_to_parse):
    soup = BeautifulSoup(html, "html.parser")
    descriptions = defaultdict(list)
    for class_name in classes_to_parse:
        for tag in soup.find_all(class_=class_name):
            descriptions[class_name].append(tag.text)
    return descriptions

In [48]:
parse_description(html_text, classes_to_parse)

defaultdict(list,
            {'ProductDescriptionHeading': ['Pulaski Furniture Living Room Two Way Sliding Door Curio 20484 at Carol House Furniture'],
             'ProductInfoSpanValue ProductInfoSpanValueProductDescription': ["This beautiful mirrored-back curio cabinet lets you display your collections: a source of pride, accomplishment, and fond memories. Give your cherished belongings the spotlight they deserve with this beautiful locking curio cabinet, fitted with a sliding beveled class door and locking mechanism for security. Five generously-sized glass shelves can be adjusted into nearly any configuration with top-to-bottom position holes every two inches, allowing you to customize the display to your liking. Along the framed edges, stunning rope-and-block molding frames and flatters the curio's contents, while two-setting halogen top-mounted lights provide an attractive, uncluttered view of your items. This curio cabinet is finished with glass sides and a fully mirrored back

In [49]:
import subprocess


def force_kill_process_by_pid(pid):
    try:
        subprocess.run(['taskkill', '/F', '/PID', str(pid)], check=True)
        print(f"Successfully terminated process PID {pid}.")
    except subprocess.CalledProcessError as e:
        print(f"Failed to terminate process PID {pid}: {e}")

In [50]:
force_kill_process_by_pid(34744)

Failed to terminate process PID 34744: Command '['taskkill', '/F', '/PID', '34744']' returned non-zero exit status 128.


In [51]:
import psutil

In [52]:
proc = psutil.Process(34744)

NoSuchProcess: process PID not found (pid=34744)